## Attribute K-S score

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col, udf
from pyspark.sql.types import *
from pyspark.sql import DataFrame
from functools import reduce


from pyspark.sql.functions import *

from pyspark.sql import Window
import pandas as pd

def ks(df, variable, target):
    df_tmp =df.filter(col(variable).isNotNull()).select(col(variable).alias('p'), col(target).alias('t'))
    return df_tmp.groupBy('p') \
                 .agg(sum('t').alias('bads'), sum(lit(1)- col('t')).alias('goods')) \
                .withColumn('cumB', 
                           sum('bads').over(
                              Window.rowsBetween(Window.unboundedPreceding, Window.curentRow) \
                                      .partitionBy() \
                                      .orderBy('p')) \
                           /sum('bads').over(Window.partitionBy())) \
                           .withColumn('cumG',
                                    sum('goods').over(
                                       Window.rowsBetween(Window.unboundedPreceding, Window.curentRow) \
                                        .partitionBy() \
                                        .orderBy('p')) \
                                   /sum('goods').over(Window.partitionBy())) \                                    
                           .groupBy() \
                           .agg(max(abs(col('cumB') - col('cumG'))).alias('value')) \
                           .select('value') \
                           .collect()[0][0]

def iv(df, variable, target):
    
    df_tmp = df.select(col(variable).alias('p'), col(target).alias('t'))
    
    bad_sum = df_tmp.select(sum('t')).collect()[0][0]
    good_sum =df_tmp.count() - bad_sum
    
    return df_tmp.groupBy('p') \
                 .agg(sum('t').alias('bads'), sum(lit(1) - col('t')).alias('goods')) \
                 .withColumn('summand',
                      (col('goods')/good_sum - col('bads')/bad_sum) * \
                            (log(col('goods')/good_sum) -log(col('bads')/bad_usm))) \
                       .groupBy() \
                       .agg(sum('summand').alias('IV'))  \
                       .select('IV') \
                       .collect()[0][0]


if __name__ == '__main__':
    spark= SparkSession.builder.getOrCreate()
    df=spark.table('')
    
    res_1=ks(df, 'signal', 'bad_flag')
    res_2=iv(df, 'signal', 'bad_flag')
    print(res_1)

## Attribute Coverage

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col, udf
from pyspark.sql.types import *
from pyspark.sql import DataFrame
from functools import reduce


from pyspark.sql.functions import *
from pyspark.sql import Window
import pandas as pd 
pd.options.display.max_rows=500
pd.options.display.max_columns=500

spark = SparkSession.builder.getorCreate() 
table_name = 'card_orgn_trin_201801v4'

#unique column to identify each record 
id_col = 'eci'
#current score column name 
score_col='v2_score_pdo'
#target column name 
target_col = 'card_target_18m'

print("data table:"+table_name)

rawl = spark.table('db crs analysis_pii_t.' + table_name) 
raw2 = reduce (DataFrame.drop, ['acct_ref_nb', 'event_year','event_month', 'eci_act_yymm', 'reg_payer_trigger_categories'], raw1)

# cast everything to double
raw3_db1 = raw2.select(*(col(c).cast (DoubleType()).alias(c) for c in raw2.columns))

# impute selected columns with an egregious number 
#rav4 = rav3_dbl.filina(9999999) 
raw4 = raw3_dbl

# Grab a list of features 
non_features = ['eci', 'target_eci_nb', 'eci_nb','trdln_tp', 're_yearmonth', 'eci_nb','year', 'month', target_col, score_col, id_col]
features_only = reduce (DataFrame.drop, non_features, raw4).columns
print("number of features: "+str(len(features_only) ) )
total=raw4.count()


coverage=raw4.select([(count(when( col(c).isNotNull() & (col(c) !=lit(0)), c))/lit(total)).alias(c) for c in features_only ])
coverage.transpose()
